In [18]:
import pandas as pd
import dynamodb
from dynamodb import DynamoTable
import json
from decimal import Decimal

In [2]:
dt = DynamoTable()

In [ ]:
dt.create_table(
    table_name='Movies',
    partition_key='year',
    partition_key_type='N',
    sort_key='title',
    sort_key_type='S',
    provisioned=True,
    rcu=10,
    wcu=10
)

In [5]:
dt.select_table('Movies')
dt

- Table name: Movies            
- Table arn: arn:aws:dynamodb:us-east-1:908988413876:table/Movies            
- Table creation: 2023-01-09 00:46:03.162000+00:00            
- [{'AttributeName': 'year', 'KeyType': 'HASH'}, {'AttributeName': 'title', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'title', 'AttributeType': 'S'}, {'AttributeName': 'year', 'AttributeType': 'N'}]

In [ ]:
dt.add_item(item)

In [14]:
# Itireate over json file and add items to table
with open('moviedata.json') as json_file:
    movies = json.load(json_file)
movies_form = json.loads(json.dumps(movies), parse_float=Decimal)

with dt.table.batch_writer() as batch:
    for movie in movies_form:
        batch.put_item(Item=movie)

In [30]:
df = dt.query_items(query=2010, to_pandas=True, consumed_capacity=True)
df.head()

Consumed Capacity: 14.5


,year,info,title
0,2010,"{'actors': ['James Franco', 'Amber Tamblyn', '...",127 Hours
1,2010,"{'actors': ['Sam Riley', 'Alice Barrett', 'Gab...",13
2,2010,"{'actors': ['Emma Roberts', 'Tamsin Egerton', ...",4.3.2.1
3,2010,"{'actors': ['Rupert Friend', 'Emmanuelle Chriq...",5 Days of War
4,2010,"{'actors': ['Jackie Earle Haley', 'Rooney Mara...",A Nightmare on Elm Street


In [36]:
dt.scan_att('title', 'The Godfather', consumed_capacity=True)

Consumed Capacity: 256.0


,year,info,title
0,1972,"{'actors': ['Marlon Brando', 'Al Pacino', 'Jam...",The Godfather


In [37]:
movie_rating = Decimal('9.2')
movie_plot = "The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son."
dt.update_item('The Godfather', 1972, movie_rating, movie_plot)

{'info': {'plot': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.',
  'rating': Decimal('9.2')}}